# Notebook 1: Random Walk e Mercados Justos

Este notebook introduz o conceito de **mercados justos** que geram random walks gaussianos.

## Objetivos de Aprendizado

1. Entender como mercados funcionam através de auction pricing
2. Observar um random walk puro em ação
3. Explorar propriedades estatísticas dos retornos

In [ ]:
# Imports
import random
from market_lab.core.market import MarketConfig
from market_lab.core.traders import build_traders
from market_lab.core.sentiment import NoSentiment
from market_lab.core.simulation import SimulationRunner
from market_lab.viz.plots import plot_price_series

import matplotlib.pyplot as plt
%matplotlib inline

## Configuração da Simulação

Vamos criar um mercado simples com 150 traders aleatórios.

In [ ]:
# Configurar mercado
config = MarketConfig(
    n_traders=150,
    initial_price=100.0,
    initial_cash=10_000.0,
    initial_holdings=100,
    min_price=50.0,
    max_price=200.0,
    n_price_points=50
)

# Criar traders
rng = random.Random(42)
traders = build_traders(config, rng, use_wealth_limit=False)

print(f"Mercado configurado com {len(traders)} traders")

In [ ]:
# Executar simulação
runner = SimulationRunner(
    config=config,
    traders=traders,
    sentiment=NoSentiment(),
    manipulator=None
)

states = runner.run(n_days=120)
print(f"Simulação concluída: {len(states)} dias")

In [ ]:
# Visualizar
plot_price_series(states)
plt.show()

## Análise Estatística dos Retornos

Vamos analisar as propriedades estatísticas dos retornos do mercado.

In [ ]:
import numpy as np

# Calcular retornos percentuais
prices = [s.price for s in states]
returns = [(prices[i] - prices[i-1]) / prices[i-1] * 100 for i in range(1, len(prices))]

# Estatísticas descritivas
mean_return = np.mean(returns)
std_return = np.std(returns)
min_return = np.min(returns)
max_return = np.max(returns)

print(f"Estatísticas dos Retornos:")
print(f"  Média: {mean_return:.4f}%")
print(f"  Desvio Padrão: {std_return:.4f}%")
print(f"  Mínimo: {min_return:.4f}%")
print(f"  Máximo: {max_return:.4f}%")

### Distribuição dos Retornos

Um random walk gaussiano deve produzir retornos normalmente distribuídos.

In [ ]:
# Visualizar distribuição dos retornos
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Histograma
ax1.hist(returns, bins=30, edgecolor='black', alpha=0.7)
ax1.set_xlabel('Retorno (%)')
ax1.set_ylabel('Frequência')
ax1.set_title('Distribuição dos Retornos Diários')
ax1.axvline(mean_return, color='red', linestyle='--', label=f'Média: {mean_return:.2f}%')
ax1.legend()

# Q-Q plot para testar normalidade
from scipy import stats
stats.probplot(returns, dist="norm", plot=ax2)
ax2.set_title('Q-Q Plot (teste de normalidade)')

plt.tight_layout()
plt.show()

# Teste de normalidade Shapiro-Wilk
statistic, p_value = stats.shapiro(returns)
print(f"\nTeste de Normalidade (Shapiro-Wilk):")
print(f"  Estatística: {statistic:.4f}")
print(f"  P-valor: {p_value:.4f}")
print(f"  Resultado: {'Normal' if p_value > 0.05 else 'Não-normal'} (α=0.05)")

### Análise de Volume

O volume de negociação também fornece insights importantes sobre a dinâmica do mercado.

In [ ]:
from market_lab.viz.plots import plot_volume_series

# Visualizar volume ao longo do tempo
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))

# Série temporal de volume
plot_volume_series(states, ax=ax1)

# Correlação preço-volume
volumes = [s.volume for s in states[1:]]  # Alinhar com retornos
ax2.scatter(returns, volumes, alpha=0.5)
ax2.set_xlabel('Retorno (%)')
ax2.set_ylabel('Volume')
ax2.set_title('Correlação Retorno vs Volume')

# Calcular correlação
correlation = np.corrcoef(returns, volumes)[0, 1]
ax2.text(0.05, 0.95, f'Correlação: {correlation:.3f}', 
         transform=ax2.transAxes, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

## Exercícios Práticos

### Exercício 1: Diferentes Configurações
Modifique os parâmetros do mercado e observe como isso afeta o random walk:
- Aumente/diminua o número de traders
- Altere a volatilidade de preço (`price_volatility`)
- Mude o volume máximo diário (`max_daily_volume`)

**Questão:** Como cada parâmetro afeta a dinâmica do mercado?

### Exercício 2: Múltiplas Simulações
Execute a simulação 10 vezes com diferentes seeds e compare:
- A distribuição dos retornos é consistente?
- O preço final varia significativamente?
- Os retornos médios são próximos de zero?

### Exercício 3: Horizonte de Tempo
Compare simulações de 30, 120 e 365 dias:
- Como o horizonte temporal afeta as estatísticas?
- A normalidade dos retornos melhora com mais dados?

### Exercício 4: Eficiência de Mercado
Um mercado eficiente não deve ter autocorrelação nos retornos. Teste isso calculando a autocorrelação dos retornos diários.

In [ ]:
# Exemplo de solução para Exercício 4: Teste de Autocorrelação
from matplotlib import pyplot as plt

# Calcular autocorrelação para diferentes lags
lags = range(1, 21)
autocorr = [np.corrcoef(returns[:-lag], returns[lag:])[0, 1] for lag in lags]

# Visualizar
plt.figure(figsize=(12, 5))
plt.bar(lags, autocorr, alpha=0.7)
plt.xlabel('Lag (dias)')
plt.ylabel('Autocorrelação')
plt.title('Autocorrelação dos Retornos Diários')
plt.axhline(y=0, color='r', linestyle='--')
plt.axhline(y=1.96/np.sqrt(len(returns)), color='gray', linestyle='--', label='95% CI')
plt.axhline(y=-1.96/np.sqrt(len(returns)), color='gray', linestyle='--')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

print("Se o mercado é eficiente, a autocorrelação deve estar dentro do intervalo de confiança.")
print("Valores fora indicam previsibilidade nos retornos.")

## Conclusões

Neste notebook, você aprendeu:

1. **Mercados como sistemas de auction pricing**: O preço de equilíbrio é determinado pela interseção de oferta e demanda
2. **Random Walk**: Mercados justos sem sentimento geram passeios aleatórios
3. **Propriedades estatísticas**: Retornos são aproximadamente normais com média zero
4. **Eficiência de mercado**: Ausência de autocorrelação indica que retornos passados não preveem futuros

### Próximos Passos

No **Notebook 2**, vamos explorar como restrições de riqueza (wealth limits) alteram a dinâmica do mercado e criam bandas de preço naturais.